## SRJ - New idea

c1 |s1> + c2|s2>

## Original adapt code

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=2) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.12299286]
Final cost: -1.4300836708403644
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [1.09716245 1.37775763]
Final cost: -1.569460383552094
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)─┤  State
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)─┤  State
2: ──X─╭FermionicDoubleExcitation(1.10)─├FermionicDoubleExcitation(1.38)─┤  State
3: ──X─├FermionicDoubleExci

In [15]:
energies =  [-1.430083670840363, -1.5694603835521481, -1.7436334426151392, -1.8421101514363758, -1.8504359363287162, -1.8594600250246982, -1.865676741081967, -1.8663879748308423, -1.8843560329745248, -1.8844361649381711, -1.8844361667007483, -1.8844364517909806, -1.8844364552896873, -1.884436575638581]


In [2]:
ash_excitation = [[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [1, 5],
 [2, 6],
 [3, 7],
 [2, 3, 6, 7],
 [0, 4],
 [0, 1, 4, 5],
 [0, 1, 6, 7],
 [2, 3, 4, 5],
 [1, 2, 5, 6],
 [0, 3, 4, 7]]

ash_excitation = ash_excitation[:13]  #Truncating the ash excitation to 12 elements
print(ash_excitation)

[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5], [2, 6], [3, 7], [2, 3, 6, 7], [0, 4], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 4, 5], [1, 2, 5, 6]]


## State 1 preparation

In [3]:
#ashs = [[2,3,6,7], [0,3,5,6], [0,1,4,5]]
#ashs = [[2,3,6,7], [0,3,5,6]]
#ashs = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [3, 7]]
#ashs = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [3, 7], [2, 6], [2, 3, 6, 7], [1, 5], [0, 1, 4, 5], [0, 1, 6, 7]]
ashs = ash_excitation
print('Len of ashs:', len(ashs))    

alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
print('Exc. going to prepare state1',ashs) 
hf_state = np.array([1,1,1,1,0,0,0,0])
qubits = 8
params = np.zeros(len(ashs), requires_grad=True)
#params = qml.numpy.array([np.pi/4], requires_grad=True)  #Initial parameters for the excitations
dev = qml.device("lightning.qubit", wires=qubits)

@qml.qnode(dev)
def s1(params, ashs, hf_state, H):
    #print('HF stat:', hf_state)
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ashs):
        #print('The value of i', i)
        if len(ashs[i]) == 4:
            #print('The ash excitation now is', ashs[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
        elif len(ashs[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

def cost(params):
    energy = s1(params, ashs, hf_state, H)
    return energy

#result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

#params = result.x
#print("Final updated parameters for new excitations:", params)
#print("Expectation value with new excitations:", result.fun)

#term1 = result.fun
#print('Term1 energy:', term1)

#State preparation after adding states and params
dev1 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev1)
def s1state(hf_state, ashs, params):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ashs):
        if len(ashs[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
        elif len(ashs[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
    return qml.state()
s1ostate = alpha * s1state(hf_state, ashs, params)
print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

Len of ashs: 13
Exc. going to prepare state1 [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5], [2, 6], [3, 7], [2, 3, 6, 7], [0, 4], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 4, 5], [1, 2, 5, 6]]
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
4: ────│────────────────────────────────│───────────────────────────────
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicSingleExcitation(0.00)
──│──────

In [4]:
ashs

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [1, 5],
 [2, 6],
 [3, 7],
 [2, 3, 6, 7],
 [0, 4],
 [0, 1, 4, 5],
 [0, 1, 6, 7],
 [2, 3, 4, 5],
 [1, 2, 5, 6]]

In [5]:
ashs[-1]

[1, 2, 5, 6]

## Generation of s2 state

In [6]:
latest_ash = [ashs[-1]]
#latest_ash = [[0,1,4,5], [0,3,4,7]]  # Next highest excitation other than [0,3,5,6]
#latest_ash = [[0,3,5,6]]
print("Latest excitation:", latest_ash)
print('Length of latest excitation:', len(latest_ash))
#params2 = np.zeros(len(latest_ash), requires_grad=True)
params2 = qml.numpy.array([np.pi/4], requires_grad=True)  # Initial parameters for the excitations
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
print(params2)
#State 2
@qml.qnode(dev)
def s2(hf_state, latest_ash, params2, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i in range(len(latest_ash)):
        print('The value of i', i)  
        if len(latest_ash[i]) == 4:
            print('The latest ash excitation in E calc is', latest_ash[i])
            qml.DoubleExcitation(params2[i], wires = latest_ash[i])  # Assuming latest_ash is a list of lists, each containing the wires for the excitation
        elif len(latest_ash[i]) == 2:
            qml.SingleExcitation(params2[i], wires = latest_ash[i]) 
    return qml.expval(H)

def cost2(params2):
    energy = s2(hf_state,latest_ash, params2, H)
    return energy
#result2= minimize(cost2, params2, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

#print("Final updated parameters for latest excitation:", result2.x)
#print("Expectation value with new excitations:", result2.fun)

#params2 = result2.x
#print('Params2 is', params2)
#term2 = result2.fun
#print('Term2 is', term2)

dev1 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev1)
def s2state(hf_state, ashs, params2):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i in range(len(latest_ash)):
        print('The value of i', i)
        if len(latest_ash[i]) == 4:
            print('The latest ash excitation is', latest_ash[i])
            print('params2 is', params2[i])
            qml.DoubleExcitation(params2[i], wires = latest_ash[i]) 
        elif len(latest_ash[i]) == 2:
            qml.SingleExcitation(params2[i], wires = latest_ash[i])
    return qml.state()

s2ostate = beta * s2state(hf_state, latest_ash, params2)
print(qml.draw(s2state, max_length=100)(hf_state,latest_ash,params2))
#print(qml.draw(s2ostate, max_length=100))



Latest excitation: [[1, 2, 5, 6]]
Length of latest excitation: 1
[0.78539816]
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
5: ────├G²(0.79)─┤  State
6: ────╰G²(0.79)─┤  State


## Numerator

In [28]:
s1ostate.shape
Ham_matrix = qml.matrix(H, wire_order=range(qubits))
s1ostatec = s1ostate.conj()

#Term1 
term1 = np.dot(s1ostatec, Ham_matrix @ s1ostate).real  #<S1|H|S1>
print(term1)
#Term2
term2 = np.dot(s2ostate.conj(), Ham_matrix @ s2ostate).real  #<S2|H|S2>
print(term2)

term3 = np.dot(s1ostatec, Ham_matrix @ s2ostate).real  #<S1|H|S2>
print(term3)
#Term4
term4 = np.dot(s2ostate.conj(), Ham_matrix @ s1ostate).real  #<S2|H|S1>
print(term4)

num = term1 + term2 + term3 + term4
print('Numerator is', num)

-1.5317520630162598
-1.5307623933299057
-1.2043282457517823
-1.2043282457517823
Numerator is -5.47117094784973


## Denominator

In [30]:
D1 = np.dot(s1ostatec,  s1ostate).real  #<S1|S1>
print(D1)
#Term2
D2= np.dot(s2ostate.conj(), s2ostate).real  #<S2|S2>
print(D2)

D3 = np.dot(s1ostatec, s2ostate).real  #<S1|S2>
print(D3)
#Term4
D4 = np.dot(s2ostate.conj(), s1ostate).real  #<S2|H|S1>
print(D4)

den = D1 + D2 + D3 + D4
print('Denominator is', den)


0.9999999999999871
1.0
0.728553390593269
0.728553390593269
Denominator is 3.4571067811865253


## Optimization

In [7]:
def global_cost(params, params2, alpha, beta):
    s1o = alpha * s1state(hf_state, ashs, params)
    s2o = beta * s2state(hf_state, latest_ash, params2)
    Ham_matrix = qml.matrix(H, wire_order=range(qubits))
    s1oc = qml.math.conj(s1o)
    s2oc = qml.math.conj(s2o)
    # Hamiltonian expectation calculations...
    term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
    #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
    term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
    #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
    term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
    #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
    term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
    numerator = term1 + term2 + term3 + term4
    #D1 = np.dot(s1oc, s1o).real
    D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
    #D2 = np.dot(s2oc, s2o).real
    D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
    #D3 = np.dot(s1oc, s2o).real
    D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
    #D4 = np.dot(s2oc, s1o).real
    D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
    denominator = D1 + D2 + D3 + D4
    return numerator / denominator

#x0 = np.concatenate([params, params2, alpha, beta])
x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

print(x0)


def global_cost_flat(x):
    params = x[:13] #0,1
    params2 = x[13:14] #2
    alpha = x[14] #3
    beta = x[15]  #4
    return global_cost(params, params2, alpha, beta)

result = minimize(global_cost_flat, x0, method='BFGS')
print(result)



[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest ash excitation is [1, 2, 5, 6]
params2 is 0.7853981633974483
The value of i 0
The latest as

In [ ]:
Ourdata = [-1.43008367, -1.56946038355194,-1.7440626583933085, -1.8660667755314557,-1.8758503116042369, -1.8835192725160552, -1.8834449071010815,  -1.883711837371623, -1.8843416301006901, -1.8843425347337301,-1.884343283547178,-1.8843437920944917,-1.8851019462629715,-1.8850866363289034]

In [17]:
ashs
latest_ash
-1.85946002
# 6 params, and [0,3,5,6] instead of last excitation - solution -1.8843425
# 6 params and last excitation - solution -1.88351927
#Adapt GCIM energy : Ground state energy is [-1.430083670535935, -1.5694603832062615, -1.7440626565880086, -1.8660667753404518, -1.8688240410155197, -1.8757124515908579, -1.8784192149543435]
#Adapt @11parameters - -1.8844361671811045

[[1, 2, 4, 7]]